In [1]:
from transformers import AutoTokenizer, BertConfig, AutoModel

import torch
from torch.utils.data import Dataset
import torch.nn as nn

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import pandas as pd



tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
config = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M")
model = AutoModel.from_config(config)

In [2]:
class CustomDataset(Dataset):
    def __init__(self, data_matrix, label_matrix):

        self.data = data_matrix
        self.labels = label_matrix

    def __len__(self):
        
        return len(self.data)

    def __getitem__(self, index):

        data_sample = self.data[index]
        label_sample = self.labels[index]

        return data_sample, label_sample
   
   
def train_model_with_dataloader(model):
    X_train, X_test, y_train, y_test = train_test_split(embeddings, labels_tensor, test_size=0.2, random_state=42)

    # Create training dataset and DataLoader
    train_data = CustomDataset(X_train, y_train)
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    # Create testing dataset and DataLoader
    test_data = CustomDataset(X_test, y_test)
    test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)


    # Training loop (assuming your data is in batches of (input, labels))
    for epoch in range(NUM_EPOCHS):
        for inputs, labels in train_dataloader:
            OPTIMIZER.zero_grad()
            outputs = model(inputs)

            # Ensure labels are of type torch.long
            labels = labels.long()

            loss = CRITERION(outputs, labels)
            loss.backward()
            OPTIMIZER.step()
            print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {loss.item():.4f}')

In [3]:
df = pd.read_csv("data/data.csv")
df.head()

,name,sequence,disease
0,hsa-miR-4458,AGAGGTAGGTGTGGAAGAA,Breast Neoplasms
1,hsa-miR-4458,AGAGGTAGGTGTGGAAGAA,Stomach Neoplasms
2,hsa-miR-4458,AGAGGTAGGTGTGGAAGAA,"Carcinoma, Non-Small-Cell Lung"
3,hsa-miR-4458,AGAGGTAGGTGTGGAAGAA,"Leukemia, Myeloid, Acute"
4,hsa-miR-4458,AGAGGTAGGTGTGGAAGAA,gastric adenocarcinoma


# embeddings

mean pooling mantıklı mı?

In [8]:
# sequences = df['sequence'].tolist()
# encoded_sequences = tokenizer(sequences, padding=True, truncation=True, return_tensors='pt')

# with torch.no_grad():
#     model_output = model(**encoded_sequences)
# embeddings = model_output.last_hidden_state.mean(dim=1)  # Assuming mean pooling for simplicity


# embedding_df = pd.DataFrame(embeddings.numpy())
# embedding_df.to_csv('data/embeddings.csv', index=False)  


embed_df = pd.read_csv('data/embeddings.csv')
embeddings = torch.tensor(embed_df.values, dtype=torch.float32)

# label encoding

In [12]:
labels = df['disease'].tolist()

# Use LabelEncoder to convert class labels to numerical class indices
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Convert numerical class indices to binary target tensor
no_of_classes = len(label_encoder.classes_)
labels_tensor = torch.zeros((len(encoded_labels), no_of_classes))
labels_tensor[range(len(encoded_labels)), encoded_labels] = 1

In [13]:
class MultiLabelClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MultiLabelClassifier, self).__init__()
        
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        output = self.sigmoid(x)
        return output
    
INPUT_SIZE = 768
HIDDEN_SIZE = 256
BATCH_SIZE = 64
NUM_EPOCHS = 10
CRITERION = nn.BCEWithLogitsLoss()
LEARNING_RATE = 0.001
OPTIMIZER = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
NUM_CLASSES = len(pd.unique(df['disease']))

model = MultiLabelClassifier(INPUT_SIZE, HIDDEN_SIZE, NUM_CLASSES)




In [14]:
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels_tensor, test_size=0.2, random_state=42)

# Create training dataset and DataLoader
train_data = CustomDataset(X_train, y_train)
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=False)

# Create testing dataset and DataLoader
test_data = CustomDataset(X_test, y_test)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)


# Training loop (assuming your data is in batches of (input, labels))
for epoch in range(NUM_EPOCHS):
    for inputs, labels in train_dataloader:
        OPTIMIZER.zero_grad()
        outputs = model(inputs)
        
        loss = CRITERION(outputs, labels)
        loss.backward()
        OPTIMIZER.step()
        print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 0.9742
Epoch [1/10], Loss: 0.9742
Epoch [1/10], Loss: 0.9742
Epoch [1/10], Loss: 0.9742
Epoch [1/10], Loss: 0.9742
Epoch [1/10], Loss: 0.9742
Epoch [1/10], Loss: 0.9742
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9742
Epoch [1/10], Loss: 0.9742
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9742
Epoch [1/10], Loss: 0.9742
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9742
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9742
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9742
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9743
Epoch [1/10], Loss: 0.9742
E

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, multilabel_confusion_matrix, classification_report
import numpy as np 


def evaluate_model(model, dataloader, criterion):
    model.eval()
    total_loss = 0.0
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            predictions = (outputs > 0.5).float()  
            all_predictions.append(predictions.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_predictions = np.concatenate(all_predictions, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average='micro')
    recall = recall_score(all_labels, all_predictions, average='micro')
    f1 = f1_score(all_labels, all_predictions, average='micro')

    avg_loss = total_loss / len(dataloader)

    print(f"Average Loss: {avg_loss:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

# Example usage:
evaluate_model(model, test_dataloader, CRITERION)


Average Loss: 0.9742
Accuracy: 0.0000
Precision: 0.0009
Recall: 0.5367
F1 Score: 0.0018


# modifying dataset

In [ ]:
# df.loc[df['disease'].str.contains('arcin|eopl|liom|oma|ancer|eukemia'), 'disease'] = 'Cancer'

# df.head()